# CNN on Cifar 10

### Import Libraries

In [30]:
import numpy as np
import pickle
import matplotlib
from matplotlib import pyplot as plt
import keras
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation, GlobalAveragePooling2D, AveragePooling2D
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

### Loading data

In [2]:
## The data is in pickle file, to unpickle it we define following function
def unpickle(file):
    with open(file,'rb') as f:
        dict_file = pickle.load(f, encoding='latin1')
    return dict_file

In [3]:
## Unpickle train and test batches.
## Get Images and their labels 
def load_data(folder_path):
    label_names = unpickle(folder_path + "\\batches.meta")['label_names']
    test_data = unpickle(folder_path + "\\test_batch")['data']
    test_labels = np.asarray(unpickle(folder_path + "\\test_batch")['labels'])
     
    data, labels = [], []
    for i in range(1,6):
        batch = unpickle(file= folder_path + "\\data_batch_" + str(i))
        if len(data) == 0:
            data = batch['data']
            labels = batch['labels']         
        else:
            data = np.vstack((data, batch['data']))
            labels = np.hstack((labels, batch['labels']))   
                        
        
    return data, labels, label_names,test_data, test_labels

train_data, train_labels, label_names, test_data, test_labels = load_data(folder_path="D:\Convolutional Neural Networks and Computer Vision\cifar-10-batches-py")

## Helper functions

In [4]:
# One hot encode labels
def one_hot(label):
    category = np.zeros((len(label), 10))
    for num, i in enumerate(label):
        category[num][i] = 1
    return category    

In [5]:
## Function for preprocessing image before feeding them algorithm
def preprocess_imgs(train, test, train_labs, test_labs):
    # Reshape Images in train and test data
    train_data = train.reshape(train.shape[0],3,32,32).transpose(0,2,3,1).astype('uint8')
    test_data = test.reshape(test.shape[0],3,32,32).transpose(0,2,3,1).astype('uint8')
    
    # Scale the image data
    train_data = train_data/ 255
    test_data  = test_data/ 255
    
    # Encode the labels
    train_labels = one_hot(train_labs)
    test_labels = one_hot(test_labs)
    
    return train_data, test_data, train_labels, test_labels

train_data, test_data, train_labels, test_labels = preprocess_imgs(train_data, test_data, train_labels, test_labels)

In [6]:
print("Shape of training data: ", train_data.shape)
print("Shape of testing data: ", test_data.shape)
print("Shape of train labels: ", train_labels.shape)
print("Shape of test labels: ", test_labels.shape)

Shape of training data:  (50000, 32, 32, 3)
Shape of testing data:  (10000, 32, 32, 3)
Shape of train labels:  (50000, 10)
Shape of test labels:  (10000, 10)


 Now that we have loaded the dataset and preprocessed them,  Images are ready for an algorithm.

In [24]:
def model():
    
    model = Sequential()
    
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(32,32,3)))    
    model.add(Conv2D(64, (3, 3), activation='relu'))    
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(10, activation='softmax'))
    
    
    return model

In [25]:
model = model()
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 30, 30, 64)        36928     
_________________________________________________________________
average_pooling2d_1 (Average (None, 15, 15, 64)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 15, 15, 64)        36928     
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 13, 13, 64)        36928     
_________________________________________________________________
average_pooling2d_2 (Average (None, 6, 6, 64)          0         
__________

Loss calculation and weight optimization for model

In [26]:
model.compile(loss='categorical_crossentropy', ##crossentropy for classification
              optimizer=Adam(lr=0.0001), ## learning rate with 1.0e-4
              metrics = ['accuracy'])

Save the best model

In [27]:
checkpoint = ModelCheckpoint('Cnn_Cifar10.h5',  # model filename
                             monitor='val_loss', # Validation loss to monitor
                             verbose=0,
                             save_best_only= True, # Saves best model only
                             mode='auto') # Automatically decides to save the best model based on 

## Train a model on training data

In [28]:
model.fit(train_data, train_labels,batch_size = 128, epochs = 100, 
          validation_data= (test_data, test_labels),verbose=1,callbacks=[checkpoint])

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 155s 3ms/step - loss: 1.8950 - acc: 0.3021 - val_loss: 1.5924 - val_acc: 0.4200
Epoch 2/100
50000/50000 [==============================] - 154s 3ms/step - loss: 1.5793 - acc: 0.4254 - val_loss: 1.4429 - val_acc: 0.4717
Epoch 3/100
50000/50000 [==============================] - 154s 3ms/step - loss: 1.4525 - acc: 0.4742 - val_loss: 1.3446 - val_acc: 0.5175
Epoch 4/100
50000/50000 [==============================] - 154s 3ms/step - loss: 1.3664 - acc: 0.5090 - val_loss: 1.2663 - val_acc: 0.5430
Epoch 5/100
50000/50000 [==============================] - 154s 3ms/step - loss: 1.3064 - acc: 0.5310 - val_loss: 1.2542 - val_acc: 0.5539
Epoch 6/100
50000/50000 [==============================] - 154s 3ms/step - loss: 1.2579 - acc: 0.5537 - val_loss: 1.1823 - val_acc: 0.5775
Epoch 7/100
50000/50000 [==============================] - 155s 3ms/step - loss: 1.2099 - acc: 0.5695 - val_loss: 1.

50000/50000 [==============================] - 154s 3ms/step - loss: 0.4906 - acc: 0.8254 - val_loss: 0.7006 - val_acc: 0.7591
Epoch 60/100
50000/50000 [==============================] - 157s 3ms/step - loss: 0.4826 - acc: 0.8297 - val_loss: 0.7115 - val_acc: 0.7596
Epoch 61/100
50000/50000 [==============================] - 155s 3ms/step - loss: 0.4672 - acc: 0.8355 - val_loss: 0.7199 - val_acc: 0.7597
Epoch 62/100
50000/50000 [==============================] - 157s 3ms/step - loss: 0.4664 - acc: 0.8359 - val_loss: 0.7171 - val_acc: 0.7547
Epoch 63/100
50000/50000 [==============================] - 159s 3ms/step - loss: 0.4544 - acc: 0.8379 - val_loss: 0.7021 - val_acc: 0.7605
Epoch 64/100
50000/50000 [==============================] - 160s 3ms/step - loss: 0.4468 - acc: 0.8397 - val_loss: 0.6983 - val_acc: 0.7657
Epoch 65/100
50000/50000 [==============================] - 159s 3ms/step - loss: 0.4443 - acc: 0.8405 - val_loss: 0.6905 - val_acc: 0.7656
Epoch 66/100
50000/50000 [=======

## Evaluating learned parameters on test set

In [29]:
test_score = model.evaluate(test_data,test_labels)
print("Test Accuracy: ", str(test_score[1] *100) + "%")

10000/10000 [==============================] - 15s 1ms/step
Test Accuracy:  76.91%
